---
title: Correlation of Enformer CAGE predictions on rn7 reference genome and measured gene expression in Br rats
author: Sabrina Mi
date: 8/23/2023

---

## Concatenate Predictions

We start with concatenating our predictions into separate files by chromosome number, firstly because our predictions are currently split between two directories and secondly, by grouping intervals together by chromosome, we can parallelize computing gene expression.

In [26]:
import glob
import os
import h5py
import parsl
from parsl import python_app
from parsl.configs.local_threads import config
parsl.load(config)


RuntimeError: Config has already been loaded

In [19]:
# intervals = [file.replace("_predictions.h5","") for file in 
#              os.listdir("/home/s1mi/Br_predictions/predictions_folder/reference_Br_all_genes/predictions_2023-08-16/enformer_predictions/reference_Br/haplotype0")]
# intervals = intervals + [file.replace("_predictions.h5","") for file in 
#              os.listdir("/home/s1mi/Br_predictions/predictions_folder/reference_Br_all_genes/predictions_2023-08-29/enformer_predictions/reference_Br/haplotype0")]
                         

In [8]:
output_dir = "/home/s1mi/Br_predictions/predictions_folder/reference_Br_all_genes"
chr_list = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "X"]

In [ ]:
## concatenate files in first prediction run
predictions_dir = f"{output_dir}/predictions_2023-08-16/enformer_predictions/reference_Br/haplotype0"
app_futures = []
for chr in chr_list:
    app_futures.append(concatenate_predictions_by_chromosome(predictions_dir, chr, output_dir))

exec_futures = [q.result() for q in app_futures]
print("First prediction folder done")

In [27]:
@python_app
def concatenate_predictions_by_chromosome(predictions_dir, chr, output_dir):
    output_path = os.path.join(output_dir, f"chr{chr}_predictions.h5")
    with h5py.File(output_path, "w") as output_file:
        files = glob.glob(f"{predictions_dir}/chr{chr}*_predictions.h5")
        for file in files:
            interval = os.path.basename(file).replace("_predictions.h5", "")
            with h5py.File(file, "r") as input_file:
                output_file[interval] = input_file[interval][()]
            